In [8]:
# Loading libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xesmf as xe
import dask
import season_util as su
import jetlatcalcs as jlat
import mask

#### Find/read sst data:

#### Calculate Nino3.4 time series:

#### Find/read pr data:

In [96]:
#historical simulations
import intake
var = ["pr"]
model=['AWI-CM-1-MR','BCC-CSM2-MR','CAMS-CSM1-0','CanESM5','MIROC-ES2L','MIROC6','UKESM1-0-LL','MRI-ESM2-0']
col = intake.open_esm_datastore("/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json")
cat = col.search(activity_id="CMIP",experiment_id="historical", variable_id=var,
table_id="Amon", grid_label="gn", source_id=model )
dset_dict_hist = cat.to_dataset_dict(cdf_kwargs={'chunks': {"time": 36}})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 7 group(s)


In [11]:
#ssp simulations
col = intake.open_esm_datastore("/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json")
cat = col.search(activity_id="ScenarioMIP",experiment_id="ssp370", variable_id=var,
table_id="Amon", grid_label="gn", source_id=model )
dset_dict_ssp370 = cat.to_dataset_dict(cdf_kwargs={'chunks': {"time": 36}})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 7 group(s)


#### Calculate monthly mean across ensembles members within each model:

In [14]:
#calculate the mean over ensemble members during desired time period
hist_emean = {}
for key, ds in dset_dict_hist.items():
    hist_mean[key] = ds.sel(time = slice('1979-01-01','2006-01-01')).mean(dim=["member_id"])
    
ssp370_emean = {}
for key, ds in dset_dict_ssp370.items():
    ssp370_mean[key] = ds.sel(time = slice('2070-01-01','2099-01-01')).mean(dim=["member_id"])

In [15]:
hist_emean = dask.compute(hist_mean)
ssp370_emean = dask.compute(ssp370_mean)

In [ ]:
## check attributes of pr variable. units are kg m-2 s-1
#dset_dict_hist['CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn'].pr

#### Calculate DJF precip time series for California:

In [108]:
hist_ca_pr = {}
for key in hist_emean[0]:
    ca_mask = mask.state_mask(mask.read_state("../data/states.shp", 'California'), hist_emean[0][key], 'California', 'CA')
    hist_ca_pr[key] = su.season_ts((hist_emean[0][key]*ca_mask), "pr", "DJF").mean(dim=('lat', 'lon'))

/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning:

In [86]:
# im not sure if we need future...if not, we can delete this
ssp370_ca_pr = {}
for key in ssp370_emean[0]:
    ca_mask = mask.state_mask(mask.read_state("../data/states.shp", 'California'), ssp370_emean[0][key], 'California', 'CA')
    ssp370_ca_pr[key] = su.season_ts((ssp370_emean[0][key]*ca_mask), var = "pr", season="DJF").mean(dim=('lat', 'lon'))

/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning:

In [114]:
# look at ssp370_ca_pr time series
#hist_ca_pr

#### Calculate interannual correlation between Nino3.4 and CA DJF precip: 

#### Calculate mean DJF precip in historical and future periods: 

In [103]:
hist_ca_mean = {}
for key in hist_emean[0]:
    ca_mask = mask.state_mask(mask.read_state("../data/states.shp", 'California'), hist_emean[0][key], 'California', 'CA')
    hist_ca_mean[key] = su.season_mean((hist_emean[0][key]*ca_mask), var="pr", season="DJF").mean(dim=('lat', 'lon'))

ssp370_ca_mean = {}
for key in ssp370_emean[0]:
    ca_mask = mask.state_mask(mask.read_state("../data/states.shp", 'California'), ssp370_emean[0][key], 'California', 'CA')
    ssp370_ca_mean[key] = su.season_mean((ssp370_emean[0][key]*ca_mask), var="pr", season="DJF").mean(dim=('lat', 'lon'))

/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning:

### Plot something: